In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [2]:
# Configuration
NUM_SESSIONS = 100000
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2024, 12, 31)

# Business rules
OVERALL_CONVERSION = 0.32  # 32% complete purchase
CART_ABANDONMENT_RATE = 0.42  # 42% abandon at cart

# Feature distributions
DEVICES = ['Desktop', 'Mobile', 'Tablet']
DEVICE_WEIGHTS = [0.45, 0.45, 0.10]

TRAFFIC_SOURCES = ['Organic Search', 'Paid Search', 'Social Media', 'Direct', 'Email']
TRAFFIC_WEIGHTS = [0.30, 0.25, 0.20, 0.15, 0.10]

PRODUCTS_VIEWED_RANGE = (1, 15)
TIME_ON_PAGE_RANGE = (10, 600)  # seconds
CART_VALUE_RANGE = (20, 500)
SHIPPING_COST_RANGE = (0, 25)

print(f"📊 Generating {NUM_SESSIONS:,} user sessions")
print(f"📅 Date range: {START_DATE.date()} to {END_DATE.date()}")

📊 Generating 100,000 user sessions
📅 Date range: 2024-01-01 to 2024-12-31


In [3]:
def generate_session_id():
    """Generate unique session ID"""
    return f"SES{np.random.randint(100000, 999999)}"

def generate_timestamp():
    """Generate random timestamp in 2024"""
    delta = END_DATE - START_DATE
    random_days = np.random.randint(0, delta.days)
    random_hours = np.random.randint(0, 24)
    random_minutes = np.random.randint(0, 60)
    
    return START_DATE + timedelta(days=random_days, hours=random_hours, minutes=random_minutes)

def calculate_abandonment_probability(device, cart_value, shipping_cost, has_discount):
    """
    Calculate cart abandonment probability based on features
    Mobile users abandon more, high shipping increases abandonment
    """
    base_prob = 0.42
    
    # Device impact
    if device == 'Mobile':
        base_prob += 0.10
    elif device == 'Tablet':
        base_prob += 0.05
    
    # Shipping cost impact (strong predictor)
    if shipping_cost > 15:
        base_prob += 0.15
    elif shipping_cost > 10:
        base_prob += 0.08
    
    # Cart value impact (inverse)
    if cart_value > 300:
        base_prob -= 0.10
    
    # Discount impact
    if has_discount:
        base_prob -= 0.12
    
    return min(max(base_prob, 0.1), 0.9)  # Clamp between 10% and 90%

print("✅ Helper functions defined")

✅ Helper functions defined


In [4]:
print("🔄 Generating user sessions...")

sessions = []

for i in range(NUM_SESSIONS):
    # Generate session attributes
    session_id = generate_session_id()
    timestamp = generate_timestamp()
    device = np.random.choice(DEVICES, p=DEVICE_WEIGHTS)
    traffic_source = np.random.choice(TRAFFIC_SOURCES, p=TRAFFIC_WEIGHTS)
    
    # User behavior
    products_viewed = np.random.randint(*PRODUCTS_VIEWED_RANGE)
    time_on_page = np.random.randint(*TIME_ON_PAGE_RANGE)
    
    # Cart attributes
    cart_value = round(np.random.uniform(*CART_VALUE_RANGE), 2)
    shipping_cost = round(np.random.uniform(*SHIPPING_COST_RANGE), 2)
    has_discount = np.random.choice([True, False], p=[0.30, 0.70])
    
    # Determine funnel progression
    # Everyone browses
    browsed = True
    
    # 85% view product
    viewed_product = np.random.random() < 0.85
    
    # 60% of viewers add to cart
    added_to_cart = viewed_product and (np.random.random() < 0.60)
    
    # Cart abandonment logic
    reached_checkout = False
    completed_purchase = False
    
    if added_to_cart:
        abandonment_prob = calculate_abandonment_probability(
            device, cart_value, shipping_cost, has_discount
        )
        
        # Did they abandon?
        abandoned = np.random.random() < abandonment_prob
        
        if not abandoned:
            reached_checkout = True
            # 90% of those who reach checkout complete purchase
            completed_purchase = np.random.random() < 0.90
    
    # Create session record
    session = {
        'session_id': session_id,
        'timestamp': timestamp,
        'device': device,
        'traffic_source': traffic_source,
        'products_viewed': products_viewed,
        'time_on_page_seconds': time_on_page,
        'cart_value': cart_value,
        'shipping_cost': shipping_cost,
        'has_discount': has_discount,
        'browsed': browsed,
        'viewed_product': viewed_product,
        'added_to_cart': added_to_cart,
        'reached_checkout': reached_checkout,
        'completed_purchase': completed_purchase
    }
    
    sessions.append(session)
    
    # Progress indicator
    if (i + 1) % 10000 == 0:
        print(f"  Generated {i + 1:,} / {NUM_SESSIONS:,} sessions...")

print("✅ Session generation complete!")

🔄 Generating user sessions...
  Generated 10,000 / 100,000 sessions...
  Generated 20,000 / 100,000 sessions...
  Generated 30,000 / 100,000 sessions...
  Generated 40,000 / 100,000 sessions...
  Generated 50,000 / 100,000 sessions...
  Generated 60,000 / 100,000 sessions...
  Generated 70,000 / 100,000 sessions...
  Generated 80,000 / 100,000 sessions...
  Generated 90,000 / 100,000 sessions...
  Generated 100,000 / 100,000 sessions...
✅ Session generation complete!


In [5]:
# Create DataFrame
df = pd.DataFrame(sessions)

print("\n📊 DATASET OVERVIEW")
print("=" * 50)
print(f"Total Sessions: {len(df):,}")
print(f"Date Range: {df['timestamp'].min().date()} to {df['timestamp'].max().date()}")
print(f"\nDataset Shape: {df.shape}")
print(f"\nColumn Names:\n{df.columns.tolist()}")
print(f"\nFirst 5 rows:")
print(df.head())


📊 DATASET OVERVIEW
Total Sessions: 100,000
Date Range: 2024-01-01 to 2024-12-30

Dataset Shape: (100000, 14)

Column Names:
['session_id', 'timestamp', 'device', 'traffic_source', 'products_viewed', 'time_on_page_seconds', 'cart_value', 'shipping_cost', 'has_discount', 'browsed', 'viewed_product', 'added_to_cart', 'reached_checkout', 'completed_purchase']

First 5 rows:
  session_id           timestamp   device  traffic_source  products_viewed  \
0  SES221958 2024-12-14 14:42:00   Mobile    Social Media               10   
1  SES164820 2024-09-14 23:43:00   Tablet  Organic Search               12   
2  SES249503 2024-07-06 15:14:00   Mobile          Direct                3   
3  SES370936 2024-11-11 13:49:00  Desktop  Organic Search                2   
4  SES560337 2024-12-25 03:01:00  Desktop  Organic Search                4   

   time_on_page_seconds  cart_value  shipping_cost  has_discount  browsed  \
0                   476       67.99          11.48         False     True   
1  

In [6]:
print("\n🔍 FUNNEL CONVERSION RATES")
print("=" * 50)

total = len(df)
browsed = df['browsed'].sum()
viewed = df['viewed_product'].sum()
added_cart = df['added_to_cart'].sum()
checkout = df['reached_checkout'].sum()
purchase = df['completed_purchase'].sum()

print(f"Browsed:           {browsed:,} ({browsed/total*100:.1f}%)")
print(f"Viewed Product:    {viewed:,} ({viewed/total*100:.1f}%)")
print(f"Added to Cart:     {added_cart:,} ({added_cart/total*100:.1f}%)")
print(f"Reached Checkout:  {checkout:,} ({checkout/total*100:.1f}%)")
print(f"Completed Purchase: {purchase:,} ({purchase/total*100:.1f}%)")

print(f"\n💰 Overall Conversion Rate: {purchase/total*100:.2f}%")
print(f"🛒 Cart Abandonment Rate: {(added_cart - checkout)/added_cart*100:.2f}%")


🔍 FUNNEL CONVERSION RATES
Browsed:           100,000 (100.0%)
Viewed Product:    84,919 (84.9%)
Added to Cart:     50,961 (51.0%)
Reached Checkout:  27,096 (27.1%)
Completed Purchase: 24,411 (24.4%)

💰 Overall Conversion Rate: 24.41%
🛒 Cart Abandonment Rate: 46.83%


In [7]:
# Save to CSV
output_path = '../data/ecommerce_funnel_data.csv'
df.to_csv(output_path, index=False)

print(f"\n✅ Dataset saved to: {output_path}")
print(f"📁 File size: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


✅ Dataset saved to: ../data/ecommerce_funnel_data.csv
📁 File size: 25.48 MB


In [1]:
# Create funnel summary for Tableau
funnel_data = pd.DataFrame({
    'Stage': ['1. Browsed', '2. Viewed', '3. Cart', '4. Checkout', '5. Purchased'],
    'Count': [
        len(df),
        df['viewed_product'].sum(),
        df['added_to_cart'].sum(),
        df['reached_checkout'].sum(),
        df['completed_purchase'].sum()
    ]
})

funnel_data.to_csv('../data/funnel_summary.csv', index=False)
print("✅ Funnel summary created!")
print(funnel_data)

NameError: name 'pd' is not defined

In [2]:
import pandas as pd

# Load data if not already loaded
try:
    len(df)  # Check if df exists
except NameError:
    df = pd.read_csv('../data/ecommerce_funnel_data.csv')
    print("✅ Data loaded")

# Create funnel summary
funnel_data = pd.DataFrame({
    'Stage': ['1. Browsed', '2. Viewed', '3. Cart', '4. Checkout', '5. Purchased'],
    'Count': [
        len(df),
        df['viewed_product'].sum(),
        df['added_to_cart'].sum(),
        df['reached_checkout'].sum(),
        df['completed_purchase'].sum()
    ]
})

# Save to CSV
funnel_data.to_csv('../data/funnel_summary.csv', index=False)

print("✅ Funnel summary created!")
print("\n📊 FUNNEL DATA:")
print(funnel_data)
print(f"\n✅ Saved to: data/funnel_summary.csv")

AttributeError: module 'pyarrow' has no attribute '__version__'